In [ ]:
from google import genai
import os

client =  genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

myfile = client.files.upload(file='/home/skamalj/dev/tataplay/separated_audio/htdemucs/media_1_audio/vocals.wav')

response = client.models.generate_content(
  model='gemini-2.5-pro',
  contents=[
    """You are given an audio clip. 
    Task:
    1. Identify the speakers.
    2. Give them indian names basis gender
    3. Descrive their speaking style

    Stye description examples:
    Aditi - Slightly High-Pitched, Expressive Tone:
    "Aditi speaks with a slightly higher pitch in a close-sounding environment. Her voice is clear, with subtle emotional depth and a normal pace, all captured in high-quality recording."

    Sita - Rapid, Slightly Monotone:
    "Sita speaks at a fast pace with a slightly low-pitched voice, captured clearly in a close-sounding environment with excellent recording quality."

    Tapan - Male, Moderate Pace, Slightly Monotone:
    "Tapan speaks at a moderate pace with a slightly monotone tone. The recording is clear, with a close sound and only minimal ambient noise."
    """,
    myfile,
  ]
)
print(response.text)

In [ ]:
from google import genai
import os

client =  genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

myfile = client.files.upload(file='/home/skamalj/dev/tataplay/separated_audio/htdemucs/media_1_audio/vocals.wav')

language = "English"
response = client.models.generate_content(
  model='gemini-2.5-pro',
  contents=[
f"""
You are given an audio clip. Perform the following tasks:

1. Transcribe the speech in its original language.
2. Translate into the target language: {language}, following these rules:
   - Preserve any code-switched words exactly (e.g., English words spoken inside Hindi).
   - Keep the translation natural and conversational.
   - Aim to match the original speech length (word count and duration) as closely as possible,
     without making the translation sound forced.
   - Ensure the translated_text remains in {language}, even if code-switched words are included.
3. Assume 3 speakers (Speaker 1, Speaker 2, Speaker 3) and perform diarization.
4. Generate SSML for each segment:
   - Use <speak> and a unique <voice> per speaker.
   - Insert <break> and <prosody> to reflect pauses, pitch, and tone.
   - Wrap the full text in a <lang xml:lang="..."> tag for {language}.
   - If a preserved word may cause mispronunciation or language switching,
     wrap it with <phoneme> or <lang> as needed.

Return strictly valid JSON in this format:

{{
  "speakers_count": <int>,
  "speak_segments": [
    {{
      "speaker": "<speaker_id>",
      "start_time": <float_seconds>,
      "end_time": <float_seconds>,
      "original_text": "<transcribed_text>",
      "translated_text": "<translation_in_{language}_with_code-switched_words_preserved>",
      "ssml": "<speak><voice name='...'><lang xml:lang='xx-XX'>...</lang></voice></speak>"
    }},
    ...
  ],
  "metadata": {{
    "translation_language": "{language}",
    "total_duration": <float_seconds>,
    "notes": "code-switched words preserved; pitch, pauses, and tone captured; translation length aligned to original"
  }}
}}

Rules:
- Every segment must include original_text, translated_text, and ssml.
- Preserve code-switched words exactly.
- Ensure translated_text is in {language} (not influenced by preserved words).
- SSML must enforce the target language (<lang xml:lang="...">).
- Keep translations concise and aligned with original timing.
- No explanation outside the JSON.
""",
    myfile
  ]
)

### Helper Function

In [ ]:
# @title Helper functions (just run that cell)

import contextlib
import wave
from IPython.display import Audio

file_index = 0

@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

def play_audio_blob(blob):
  global file_index
  file_index += 1

  fname = f'audio_{file_index}.wav'
  with wave_file(fname) as wav:
    wav.writeframes(blob.data)

  return Audio(fname, autoplay=True)

def play_audio(response):
    return play_audio_blob(response.candidates[0].content.parts[0].inline_data)

In [ ]:
import json

# Example: Gemini response text
text = response.text

# Remove leading ```json and trailing ```
if text.startswith("```json"):
    text = text[len("```json"):].strip()
if text.endswith("```"):
    text = text[:-3].strip()

# Now parse as JSON
gemini_data = json.loads(text)

# save gemini_data to a file
with open('translated_diarized_data.json', 'w', encoding='utf-8') as f:
    json.dump(gemini_data, f, indent=2,ensure_ascii=False)


In [ ]:
def create_wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

In [ ]:
from google.genai import types
from pathlib import Path

def synthesize_segment_tts(client, segment, output_dir="translated_segments", voice_name="Puck"):
    """
    Generate TTS audio for a single segment, keeping duration for lip-sync.

    segment: dict with keys
        - ssml: SSML string
        - start_time: float (seconds)
        - end_time: float (seconds)
        - speaker: string
    client: gemini client
    output_dir: where to save the segment audio
    voice_name: prebuilt Gemini TTS voice
    """
    import os
    Path(output_dir).mkdir(exist_ok=True)
    
    # Calculate duration from segment metadata
    duration = float(segment["end_time"])  - float(segment["start_time"])
    
    # Prepare output file path
    seg_index = segment.get("index", 0)

    speaker = segment.get("speaker", "Speaker")
    out_file = Path(output_dir) / f"{seg_index}_{speaker.replace(' ', '_')}.wav"

    print(f"Processing segment {seg_index} with duration {duration:.3f} seconds to {out_file}")
    print(segment['translated_text'])
    # Wrap the SSML into the TTS prompt
    tts_prompt = f"""
    You are given SSML with timing metadata. 
    Identify the target language from text. If language is english ensure that accent is **Indian English**.
    Must keep original pauses and restrict **durations to {duration:.3f} seconds**. 
    Generate **audio output**.

    Here is the SSML:

    {segment['translated_text']}
    """
    #gemini-2.5-flash-preview-tts
    #emini-2.5-pro-preview-tts
    response = client.models.generate_content(
        model='gemini-2.5-pro-preview-tts',
        contents=tts_prompt,
        config=types.GenerateContentConfig(
            response_modalities=["AUDIO"],
            speech_config=types.SpeechConfig(
                voice_config=types.VoiceConfig(
                    prebuilt_voice_config=types.PrebuiltVoiceConfig(
                        voice_name=voice_name,
                    )
                )
            ),
        )
    )
    print(response.candidates[0])
    # Save the returned audio bytes
    if response.candidates[0].content:
        data = response.candidates[0].content.parts[0].inline_data.data # Gemini returns audio bytes in response.audio
        create_wave_file(str(out_file), data)


In [ ]:
!rm translated_segments/*.wav

### Block is only for testing. Run for any single segment

In [ ]:
n = 3  # change this index as needed
seg = gemini_data["speak_segments"][n]
seg["index"] = n

synthesize_segment_tts(client, seg, voice_name="Puck")


In [ ]:
from google import genai
import os
client =  genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

In [ ]:
# Assume `gemini_data` is your parsed Gemini JSON with speak_segments
for idx, seg in enumerate(gemini_data["speak_segments"]):
    seg["index"] = idx
    synthesize_segment_tts(client, seg, voice_name="Puck")


## Code to stretch a audio for segments. WIP

In [1]:
from pydub import AudioSegment
from pydub.silence import detect_silence
from audiostretchy.stretch import stretch_audio

def remove_silence_from_end(audio_path, output_path, silence_thresh=-50, chunk_size=10):
    audio = AudioSegment.from_file(audio_path)
    silence_ranges = detect_silence(audio, min_silence_len=chunk_size, silence_thresh=silence_thresh)

    # Default end_trim is full length of audio
    end_trim = len(audio)

    # Check if last silence segment is at the end of audio
    if silence_ranges and silence_ranges[-1][1] == len(audio):
        # Trim silence at the end
        end_trim = silence_ranges[-1][0]

    # Keep audio from start to end_trim (remove silence only at end)
    trimmed_audio = audio[:end_trim]
    trimmed_audio.export(output_path, format="wav")

# Usage example:
#remove_silence_from_end("translated_segments/1_Speaker_2_compress.wav", "translated_segments/trimmed_end_only.wav")


In [2]:
import json
from pathlib import Path
from pydub import AudioSegment
import soundfile as sf
from audiostretchy.stretch import stretch_audio

# Paths
json_path = "translated_diarized_data.json"
tts_dir = Path("translated_segments")          # TTS-generated segments
adjusted_dir = Path("adjusted_segments")      # Save adjusted segments
adjusted_dir.mkdir(exist_ok=True)

# Load JSON
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

segments = data["speak_segments"]


for idx, seg in enumerate(segments):
    speaker = seg.get("speaker", "Speaker")
    
    tts_file = tts_dir / f"{idx}_{speaker.replace(' ', '_')}.wav"
    if not tts_file.exists():
        print(f"Missing file: {tts_file}")
        continue

    # Target duration from JSON
    target_duration = float(seg["end_time"]) - float(seg["start_time"])  # in seconds

   # Load your audio file
    audio = AudioSegment.from_file(tts_file)

    # Get the duration in milliseconds
    current_duration = audio.duration_seconds

    # Save adjusted segment
    out_file = adjusted_dir / f"adjusted_seg_{idx}_{seg['speaker'].replace(' ', '_')}.wav"
    out_file_stretched = adjusted_dir / f"adjusted_seg_{idx}_{seg['speaker'].replace(' ', '_')}_stretched.wav"

    stretch_factor = max(0.7, min(1.3, target_duration / current_duration))
    adjusted_audio = stretch_audio(tts_file, out_file, ratio=stretch_factor)
    remove_silence_from_end(out_file, out_file_stretched)

    print(f"Saved adjusted segment: {out_file} (target: {target_duration:.2f}s, actual: {current_duration:.2f}s)")


Saved adjusted segment: adjusted_segments/adjusted_seg_0_Speaker_1.wav (target: 1.13s, actual: 1.25s)
Saved adjusted segment: adjusted_segments/adjusted_seg_1_Speaker_1.wav (target: 2.05s, actual: 3.29s)
Saved adjusted segment: adjusted_segments/adjusted_seg_2_Speaker_1.wav (target: 5.02s, actual: 6.45s)
Saved adjusted segment: adjusted_segments/adjusted_seg_3_Speaker_2.wav (target: 4.52s, actual: 4.85s)
Saved adjusted segment: adjusted_segments/adjusted_seg_4_Speaker_2.wav (target: 0.79s, actual: 2.45s)
Saved adjusted segment: adjusted_segments/adjusted_seg_5_Speaker_1.wav (target: 2.43s, actual: 3.65s)
Saved adjusted segment: adjusted_segments/adjusted_seg_6_Speaker_2.wav (target: 4.13s, actual: 4.09s)
Saved adjusted segment: adjusted_segments/adjusted_seg_7_Speaker_3.wav (target: 1.87s, actual: 3.21s)
Saved adjusted segment: adjusted_segments/adjusted_seg_8_Speaker_3.wav (target: 3.24s, actual: 4.81s)


### Switch to OpenVoice Environment

In [1]:
import json
from pathlib import Path

# ----------- Load Gemini segments -----------
with open("translated_diarized_data.json", "r", encoding="utf-8") as f:
    gemini_data = json.load(f)

segments = gemini_data.get("speak_segments", [])

print(f"Loaded {len(segments)} segments from file")

Loaded 9 segments from file


In [5]:
import torch
from openvoice.api import ToneColorConverter

ckpt_converter = 'checkpoints_v2/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

/home/skamalj/.conda/envs/openvoice/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [6]:
import re
from pathlib import Path
from collections import defaultdict

from pydub import AudioSegment
import torch
import warnings

# OpenVoice se_extractor import (make sure OpenVoice is on PYTHONPATH)
from openvoice import se_extractor

def _sanitize_speaker(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9_\-]", "_", s).strip("_")

def build_reference_embeddings_from_diarization(
    original_audio_path: str,
    gemini_data: dict,
    out_dir: str = "reference_segments",
    join_silence_ms: int = 100,
    sample_rate: int = 24000,
    min_total_duration_sec: float = 2.5,
    tone_color_converter=tone_color_converter,
):
    """
    Concatenate all segments for each speaker (in chronological order),
    export a single WAV reference per speaker, then create + save OpenVoice embeddings.

    Returns:
      dict mapping speaker -> {"ref_wav": str, "embedding": str, "duration_s": float}
    """

    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Load full original audio once (pydub uses ffmpeg behind the scenes)
    audio = AudioSegment.from_file(str(original_audio_path))
    full_duration_ms = len(audio)
    print(f"[info] Loaded original audio '{original_audio_path}' ({full_duration_ms/1000:.2f}s)")

    # Group segments by speaker (keeps order)
    speaker_segments = defaultdict(list)
    for seg in gemini_data.get("speak_segments", []):
        try:
            start_ms = int(round(float(seg["start_time"]) * 1000))
            end_ms   = int(round(float(seg["end_time"]) * 1000))
        except Exception as e:
            print(f"[warning] skipping segment with bad times: {seg} -> {e}")
            continue
        if end_ms <= start_ms:
            print(f"[warning] skipping zero/negative-length segment: start={start_ms} end={end_ms}")
            continue
        # clamp within audio
        start_ms = max(0, min(start_ms, full_duration_ms))
        end_ms   = max(0, min(end_ms, full_duration_ms))
        dur_ms = end_ms - start_ms
        speaker = seg.get("speaker", "unknown").replace(' ', '_')
        speaker_segments[speaker].append({"start_ms": start_ms, "end_ms": end_ms, "duration_ms": dur_ms})

    results = {}

    for speaker, segs in speaker_segments.items():
        # sort by start time to keep natural order
        segs_sorted = sorted(segs, key=lambda x: x["start_ms"])
        concat = AudioSegment.silent(duration=0, frame_rate=sample_rate)

        total_ms = 0
        for s in segs_sorted:
            start_ms = s["start_ms"]
            end_ms = s["end_ms"]
            # slice original audio
            clip = audio[start_ms:end_ms]
            # append clip
            concat += clip
            total_ms += len(clip)
            # add tiny silence between clips to avoid gluing words (optional)
            concat += AudioSegment.silent(duration=join_silence_ms, frame_rate=sample_rate)

        total_s = total_ms / 1000.0

        # Warn if too short
        if total_s < min_total_duration_sec:
            warnings.warn(
                f"[warn] total concatenated duration for speaker '{speaker}' is short ({total_s:.2f}s). "
                "Embeddings may be poor. Consider merging more segments or increasing min_total_duration_sec."
            )

        # Trim trailing silence
        # If concat is longer than join silence at end, remove last join_silence_ms
        if len(concat) >= join_silence_ms:
            concat = concat[:-join_silence_ms]

        # Normalize export settings
        concat = concat.set_frame_rate(sample_rate).set_channels(1)

        # Save reference wav
        speaker_safe = _sanitize_speaker(speaker)
        ref_path = out_dir / f"ref_{speaker_safe}.wav"
        concat.export(str(ref_path), format="wav")
        print(f"[ok] Saved ref audio for '{speaker}' -> {ref_path} ({total_s:.2f}s)")

        # Create embedding via se_extractor
        try:
            # se_extractor.get_se may accept different signatures; attempt with tone_color_converter first
            if tone_color_converter is not None:
                source_se, audio_name = se_extractor.get_se(str(ref_path), tone_color_converter, vad=True)
            else:
                # fallback signature
                source_se, audio_name = se_extractor.get_se(str(ref_path), vad=True)
        except TypeError:
            # try alternate call (some builds expect only path)
            source_se, audio_name = se_extractor.get_se(str(ref_path))

        emb_path = out_dir / f"{speaker_safe}_embedding.pt"
        torch.save(source_se, str(emb_path))
        print(f"[ok] Saved embedding for '{speaker}' -> {emb_path}")

        results[speaker] = {
            "ref_wav": str(ref_path),
            "embedding": str(emb_path),
            "duration_s": total_s,
            "num_segments": len(segs_sorted),
        }

    return results


In [7]:
# gemini_data = json.load(open("translated_diarized_data.json", encoding="utf-8"))
# original_audio = "original_audio.mp3"

ref_map = build_reference_embeddings_from_diarization(
    original_audio_path="media_1_audio.mp3",
    gemini_data=gemini_data,
    out_dir="reference_segments",
    join_silence_ms=100,
    sample_rate=24000,
    min_total_duration_sec=5.0,
    tone_color_converter=tone_color_converter  # or None
)

print(ref_map)
# -> {'Speaker 1': {'ref_wav': 'reference_segments/ref_Speaker_1.wav', 'embedding': 'reference_segments/Speaker_1_embedding.pt', ...}, ...}


[info] Loaded original audio 'media_1_audio.mp3' (35.11s)
[ok] Saved ref audio for 'Speaker_1' -> reference_segments/ref_Speaker_1.wav (10.63s)
OpenVoice version: v2
[(0.11, 6.642), (6.83, 10.93)]
after vad: dur = 10.632


/home/skamalj/.conda/envs/openvoice/lib/python3.9/site-packages/torch/functional.py:730: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /pytorch/aten/src/ATen/native/SpectralOps.cpp:875.)
  return _VF.stft(  # type: ignore[attr-defined]


[ok] Saved embedding for 'Speaker_1' -> reference_segments/Speaker_1_embedding.pt
[ok] Saved ref audio for 'Speaker_2' -> reference_segments/ref_Speaker_2.wav (9.44s)
OpenVoice version: v2
[(0.0, 9.64)]
after vad: dur = 9.64
[ok] Saved embedding for 'Speaker_2' -> reference_segments/Speaker_2_embedding.pt
[ok] Saved ref audio for 'Speaker_3' -> reference_segments/ref_Speaker_3.wav (5.11s)
OpenVoice version: v2
[(0.0, 5.21)]
after vad: dur = 5.21
[ok] Saved embedding for 'Speaker_3' -> reference_segments/Speaker_3_embedding.pt
{'Speaker_1': {'ref_wav': 'reference_segments/ref_Speaker_1.wav', 'embedding': 'reference_segments/Speaker_1_embedding.pt', 'duration_s': 10.63, 'num_segments': 4}, 'Speaker_2': {'ref_wav': 'reference_segments/ref_Speaker_2.wav', 'embedding': 'reference_segments/Speaker_2_embedding.pt', 'duration_s': 9.44, 'num_segments': 3}, 'Speaker_3': {'ref_wav': 'reference_segments/ref_Speaker_3.wav', 'embedding': 'reference_segments/Speaker_3_embedding.pt', 'duration_s': 5.1

In [9]:
from pydub import AudioSegment
import torch
from pathlib import Path

def build_src_embedding_from_tts(tts_dir: str, gemini_data: dict, tone_color_converter=None, max_len_sec=10):
    """
    Combine TTS segments (up to max_len_sec) to create one source embedding.
    """
    combined = AudioSegment.silent(duration=0)
    total_ms = 0

    
    for idx, seg in enumerate(gemini_data["speak_segments"]):
        speaker = seg.get("speaker", "Speaker").replace(' ', '_')
        tts_path = Path(tts_dir) / f"{idx}_{speaker}.wav"
        if not tts_path.exists():
            continue

        seg_audio = AudioSegment.from_wav(tts_path)
        combined += seg_audio
        total_ms += len(seg_audio)

        if total_ms >= max_len_sec * 1000:
            break

    # Save temporary combined TTS clip
    tmp_path = Path(tts_dir) / "tts_src_reference.wav"
    combined.export(tmp_path, format="wav")

    # Extract embedding once
    if tone_color_converter is not None:
        src_se, _ = se_extractor.get_se(str(tmp_path), tone_color_converter, vad=True)
    else:
        src_se, _ = se_extractor.get_se(str(tmp_path), vad=True)

    print(f"[ok] Source embedding created from {total_ms/1000:.1f}s of TTS audio -> {tmp_path}")
    return src_se, str(tmp_path)


In [10]:
# Step 2: Build **single src embedding** from TTS segments
src_se, tts_ref_path = build_src_embedding_from_tts(
    tts_dir="translated_segments",
    gemini_data=gemini_data,
    tone_color_converter=tone_color_converter
)

OpenVoice version: v2
[(0.0, 10.992875)]
after vad: dur = 10.992
[ok] Source embedding created from 11.0s of TTS audio -> translated_segments/tts_src_reference.wav


In [11]:
def convert_tts_segments_with_refs(tts_dir, gemini_data, ref_map, out_dir, src_se, tone_color_converter):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    results = {}
    for idx, seg in enumerate(gemini_data.get("speak_segments", [])):
        speaker = seg["speaker"].replace(' ', '_')

        tts_wav = Path(tts_dir) / f"adjusted_seg_{idx}_{speaker}_stretched.wav"
        if not tts_wav.exists():
            print(f"[skip] No TTS audio for seg {idx}")
            continue

        if speaker not in ref_map:
            print(f"[skip] No reference for {speaker}")
            continue

        tgt_se = torch.load(ref_map[speaker]["embedding"])
        out_wav = out_dir / f"converted_seg_{idx}_{speaker}.wav"

        tone_color_converter.convert(
            audio_src_path=str(tts_wav),
            src_se=src_se,   # <-- reuse the same source embedding
            tgt_se=tgt_se,
            output_path=str(out_wav),
            message="@MyShell",
        )
        print(f"[ok] Converted seg {idx} ({speaker}) -> {out_wav}")
        results[idx] = str(out_wav)

    return results


In [12]:
converted = convert_tts_segments_with_refs(
    tts_dir="adjusted_segments",
    gemini_data=gemini_data,
    ref_map=ref_map,
    out_dir="converted_segments",
    src_se=src_se,
    tone_color_converter=tone_color_converter
)

Audio too short, fail to add watermark
[ok] Converted seg 0 (Speaker_1) -> converted_segments/converted_seg_0_Speaker_1.wav
[ok] Converted seg 1 (Speaker_1) -> converted_segments/converted_seg_1_Speaker_1.wav
[ok] Converted seg 2 (Speaker_1) -> converted_segments/converted_seg_2_Speaker_1.wav
[ok] Converted seg 3 (Speaker_2) -> converted_segments/converted_seg_3_Speaker_2.wav
Audio too short, fail to add watermark
[ok] Converted seg 4 (Speaker_2) -> converted_segments/converted_seg_4_Speaker_2.wav
[ok] Converted seg 5 (Speaker_1) -> converted_segments/converted_seg_5_Speaker_1.wav
[ok] Converted seg 6 (Speaker_2) -> converted_segments/converted_seg_6_Speaker_2.wav
Audio too short, fail to add watermark
[ok] Converted seg 7 (Speaker_3) -> converted_segments/converted_seg_7_Speaker_3.wav
[ok] Converted seg 8 (Speaker_3) -> converted_segments/converted_seg_8_Speaker_3.wav


In [13]:
import json
from pathlib import Path
from pydub import AudioSegment

def mix_segments_with_background(gemini_json_path, converted_dir, background_audio_path, output_path):
    # Load segment metadata
    with open(gemini_json_path, "r", encoding="utf-8") as f:
        gemini_data = json.load(f)

    speak_segments = gemini_data["speak_segments"]

    # Load background audio
    bg_audio = AudioSegment.from_file(background_audio_path, format="mp3")
    
    # Overlay each converted segment
    for idx, seg in enumerate(speak_segments):
        speaker = seg["speaker"].replace(' ', '_')
        seg_file = Path(converted_dir) / f"converted_seg_{idx}_{speaker}.wav"
        if not seg_file.exists():
            print(f"Skipping missing segment: {seg_file}")
            continue
        
        segment_audio = AudioSegment.from_file(seg_file, format="wav")
        start_ms = int(float(seg["start_time"]) * 1000)
        bg_audio = bg_audio.overlay(segment_audio, position=start_ms)
        print(f"Overlayed segment {idx} ({seg['speaker']}) at {start_ms} ms")

    # Export final mix
    bg_audio.export(output_path, format="wav")
    print(f"Final audio mix saved to {output_path}")

# Example usage
mix_segments_with_background(
    gemini_json_path="translated_diarized_data.json",
    converted_dir="converted_segments",
    background_audio_path="separated_audio/htdemucs/media_1_audio/no_vocals.wav",
    output_path="final_mix.wav"
)


Overlayed segment 0 (Speaker 1) at 0 ms
Overlayed segment 1 (Speaker 1) at 1810 ms
Overlayed segment 2 (Speaker 1) at 4850 ms
Overlayed segment 3 (Speaker 2) at 9980 ms
Overlayed segment 4 (Speaker 2) at 15220 ms
Overlayed segment 5 (Speaker 1) at 16290 ms
Overlayed segment 6 (Speaker 2) at 19460 ms
Overlayed segment 7 (Speaker 3) at 26310 ms
Overlayed segment 8 (Speaker 3) at 28610 ms
Final audio mix saved to final_mix.wav


In [1]:
from moviepy import VideoFileClip, AudioFileClip

# Load video and audio clips
video = VideoFileClip("media_1_video.mp4")
audio = AudioFileClip("final_mix.wav")

# Set the new audio to the video
final_video = video.with_audio(audio)

# Export the combined file
final_video.write_videofile("output_video_with_audio.mp4")


MoviePy - Building video output_video_with_audio.mp4.
MoviePy - Writing audio in output_video_with_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video output_video_with_audio.mp4



MoviePy - Done !
MoviePy - video ready output_video_with_audio.mp4
